# Hello and welcome to the Mabtech Iris Data Sheet Creator

INTRO: 
This script works specifically for the Mabtech Iris machine's folder output. At the end of this script you should have a Master Excel file that summarizes the results within all of your selected IRIS folders. The script currently takes in triplicates and produces an average, standard deviation, SFC per million, SI, Poisson per well in triplicate, p-value (based on T test), and determines significance based on these. If any errors come up that the script does not address, please let me know: jmorgan@lji.org. 

BEFORE YOU START:
The script is based on particular assumptions about your Fluorospot Plate layout. Please refer to the included Word document titled "Python-Mabtech Iris Data Sheet Creator Plate Layout" if you are unsure whether your plate will work. As you scroll through the script, you will notice light blue lines led by #s within each box. These are instructions or explanations of what each line of code is doing. If you are interested in changing the script, you can refer to the #s to gleam the purpose of each line. Otherwise, the only absolutely necessary box to pay attention to is box #2 discussed below.

INPUT NEEDED BY YOU:
In order for this script to work correctly, please read and follow the instructions following the #s in box #2. If the sentence is cut off by the box, scroll to the left to continue reading the instructions.

Once you have filled in the appropriate information in box #2, scroll up and look for a drop down menu labelled "Cell" and click it, then scroll down to "Run All" and click it. You should now see an * in place of the number within the brackets to the left of each box. This means the script is running. When the * is replaced by a number, that means that particular box has finished running. If the script has an error pop up after the * turns to a number, refer to the next paragraph.

ERROR HANDLING:
There are multiple types of errors that can occur. When an error occurs, the box that the error occurs in will have a number within the brackets to the left of the box and the following box will have an empty pair of brackets. 

Most likely this error has to do with wording, so first look back at your inputs in box #2 and double check that the spelling and capitilization correctly reflects your folders names. If nothing seems to be out of order here, refer to the #s within the problematic box and see if there are any instructions on how to fix the error. Lastly, you can try and google the error that pops up after the box fails to run (the last line in red is all that is needed to look it up). 

One thing to note: if you change something within a box AFTER you ran the box prior, you will have to RErun all the boxes after the changed box to see a changed result.

If an error occurs that you cannot solve using the error handling methods listed above, please email me the issue and I will do my best to help you.

In [45]:
#1
#Always run this box first before trying to run any other boxes. 
#Only needs to be run once per session.
import os
import pandas as pd
import collections

In [46]:
#2
#This box will contain pop-ups that need to be answered
#The first pop-up will ask the number of donors used PER PLATE. 
#Please click in the box pop-up below and type the number used (use numbers, dont spell it out) then click enter
#From the number you responded with, multiple pop-ups will come up (one at a time)
#Please input the exact well locations of the negatives for the FIRST DONOR, then the next well locations for the following donor (etc)
#To type in the well locations, include the row letter capitalized and the column number, seperate the individual well locations with a comma without a space
#Example: 4 donors with all the negatives in the last row 
#Type "4" (without quotes) in the first pop-up then click enter.
#Another pop-up will come up, type in the well locations for the first donor, "H1,H2,H3" (without quotes), then press enter
#Another pop-up will come up, type int he well locations for the second donor, "H4,H5,H6" (without quotes), then press enter
#Repeat until all the donor's negative locations appear below.
#If a mistake was made and you already entered the mistake, re-run this box

#This is an object that takes in a user's input and makes it into an integer-type object
number_of_donors = int(input("Please enter the number of donors ran on single fluorospot plate: "))

#Lists of the responses that will be manipulated later
list_of_negs=[]
Master_neg_list= []
donor_name_list=[]

#This i will be used as a starting location for the while loop
i=0

#This if statement is just to catch a mistake number of donors (0 or negative numbers)
if number_of_donors > 0:
    
    #The while loop is used to enter multiple negative locations based on the number of donors inputed
    #Essentially i divided by the number inputed will only be equal to 1 when they're the same number
    #So this loop starts at 0 then counts up everytime the variable is not equal to one
    while i/number_of_donors != 1: 
        
        #Another input variable that will be the filled in with each negative well location per donor
        #This will pop up each time the well loop runs, so for the # of donors used per plate
        negative_control=list(input("Please enter the exact well locations of the negative wells for one donor (note: capitalize letter and seperate individual wells with a comma and no space): ").split(","))
        
        #Save to list
        list_of_negs.append(negative_control)
        
        #Only saving last well location to this list
        Master_neg_list.append(negative_control[-1])
        
        #Creating a special donor name per negative list
        donor_name="donor"+str((i+1))
        
        #Saving donor name
        donor_name_list.append(donor_name)
        
        #Once the while loop is ran, this increases i by one so it will count up
        i=i+1
else:
    print("DONOR INPUT INVALID: must be 1 or more")
    
flat_neg_list = [item for sublist in list_of_negs for item in sublist]

Please enter the number of donors ran on single fluorospot plate: 1
Please enter the exact well locations of the negative wells for one donor (note: capitalize letter and seperate individual wells with a comma and no space): H10,H11,H12


In [47]:
#3
#This is the most complicated box, but it is necessary.
#Please fill in the below variables, all that is needed for you to change will be on the RIGHT side of equals sign.
#Once filled in, run this box.

#a
#Create the path to wherever your Iris folders or Iris excel sheets are. 
#Pay close attention to capitalization and spaces.
#Use / to indicate the next step in the path.
#The starting location of the path is the next step after the location of this program.
#So if you dragged this program, "Mabtech Iris Data Sheet Creator," to your Desktop, "Desktop" WOULD NOT be used below.
#If you dragged this program into a folder, say a master folder that contains other folders of data, that folder name WOULD NOT be included below.
#The example below goes into a folder labelled "Python for Public", then ends in the folder "DATA" which contains all the iris output folders.

folder_path = "TB Deconvolution One Negative"

#b
#Fill in the number of cells you plated per well using numbers (don't spell out number).
#DO NOT use quotes.
#In the example below, PBMCs are plated at 50uL per well starting from 4 million cells per mL concentration.

Number_of_cells_per_well = 200000

#c
#Fill in the minimum number of sfc hits acceptable to be considered a positive hit.
#In the example below, positive triplicates are only considered significant in the last column if their net sfcs are above 20, otherwise it is considered negative.

Number_of_sfcs_necessary_to_be_considered=20

#d
#This is the blank master excel sheet that will be filled out through the script.
#Fill in the location (same way as folder_path above was filled in) and name what you want the end excel file to be.
#Only fill in what is between the first quotation marks, do not change the engine.
#If this program is located on your desktop and you want the resulting excel file to be located there, just create a name for the excel file within the quotes.
#Make sure to END WITH .xlsx to your title name, otherwise it will not appear.

writer = pd.ExcelWriter('TB_Decon_ME_1Neg.xlsx', engine='xlsxwriter')

In [48]:
#4
#This creates a directory of the iris folders using the path made above.
Mabtech_Iris_Folders_dir = os.listdir(folder_path) 

#This empty list will be filled with your data.
df_list = []
error_issue_list1=[]
error_issue_list2=[]

#This begins a loop searchig through the folder you labelled in the path.
for option in Mabtech_Iris_Folders_dir:
   
    #This is looking at the folders/files in the path and checking for excel files, if it finds them it will continue to the next step.
    if option.endswith(".xlsx"):
        
        try:
            #If excel files are found, it will create a DataFrame with the data found on sheet 2 of the mabtech excel files.
            path_new = folder_path + "/" + option
            Excel_Data=pd.read_excel(path_new,sheet_name=1, usecols="A:H")

            #This adds the Dataframe to the empty list.
            df_list.append(Excel_Data)
        
        #This will add any files with issues (corrupted) into a folder that will be mentioned later
        except:
            error_issue_list1.append(option)
   
    #If no excel files are found, this will go into the folders within the path and look for excel files.
    else:
        
        #This avoids hidden files to prevent them from breaking the code.
        if not option.startswith("."):
            excel_files_in_folder = []
            for file in os.listdir(folder_path+"/"+option):
                if file.endswith(".xlsx"):
                    try:
                        #After going into the next folder looking for the excel file, it will create a Dataframe from the found excel.
                        excel_files_in_folder.append(file)
                        path_new = folder_path + "/" + option + "/" + file
                        Excel_Data=pd.read_excel(path_new,sheet_name=1, usecols="A:H")

                        #This adds the Dataframe to the empty list
                        df_list.append(Excel_Data)
                    except:
                        error_issue_list2.append(file)
                    
            #Because Mabtech Iris Folders only contain one excel per output, I incorporated this check to ensure only one excel file appears within the folder.
            if len(excel_files_in_folder) != 1:
                print("Check the following folder(s) for duplicate excel files: " + option)
                
#These print out if issues were found and the number of functional files found
print(str(len(df_list))+" functional excel files found and added to dataframe")
print(str(len(error_issue_list1))+" disfunctional excel files found in named file list (first for loop in this box): "+ str(error_issue_list1))
print(str(len(error_issue_list2))+" disfunctional excel files found in files in named file list (second for loop in this box): "+str(error_issue_list2))

21 functional excel files found and added to dataframe
0 disfunctional excel files found in named file list (first for loop in this box): []
0 disfunctional excel files found in files in named file list (second for loop in this box): []


In [49]:
#5
#This box defines a function that creates a list of headers within a dataframe list.
#It will be used as a simple check to make sure the correct headers were read in to a dataframe.
def check_headers(df_list):
    Col_Names=[]
    
    #Start of a for loop that will look through the newly created list.
    for dataframes in df_list:
        
        #This creates an object, x, which contains the list of column headers.
        x=list(dataframes.columns)
        Col_Names.append(x)
        
    #This ends the function and tells it what to output when it is ran (in this case the list of headers).    
    return(Col_Names)

In [50]:
#6
#This function is needed to run unequal columns together.
#If you have multiple Negative triplicates, this will ensure the longer row wont cause a break in the script.
def blank_filler(new_df_row):
    
    #This next section creates a loop that fills in blanks (NaN) until the new df row is equal to the New_col list.
    while ( len(new_df_row) < len( New_col) ):
        
        #The word None is used here to fill in NaN variables that python will recognize as null data rather than a 0.
        #This is imprtant bc if anything else is used (string-wise) the column will no longer be seen as a data column by python.
        new_df_row.append(None)
        
    return (new_df_row)

In [51]:
#7
#This creates a basic Dataframe that includes triplicates based on the well.

#This creates an empty dictionary which will be used to associate triplicates to wells.
letter_dict = {}
letter_str='ABCDEFGH'

#This for loop associates individual letters to a number that will be used to call on its location.
for i in range(0,len(letter_str)):
    letter = letter_str[i]
    letter_dict[i]=letter

well_list=[]
trip_marker_list=[]

#This for loop starts to number the 96 well plate.
for well_number in range(0,96):    
    col_number = (well_number % 12) + 1

    row_index = int(well_number/12)
    row_letter = letter_dict[row_index]

    #Calculate triplicates as if numbered across the plate.
    trip_across = int(well_number/3) + 1
    row_offset = -(row_index*3)
    col_offset = int((col_number-1)/3)*7

    #Add row & column offsets to the triplicate value (across) to find triplicate value when going down the plate.
    trip_down = (trip_across + row_offset + col_offset)
    
    #The well now matches the Well output from Iris.
    well=row_letter+ str(col_number)

    well_list.append(well)
   
    trip_marker_list.append(trip_down)

    
#This is the DF that will be used to create a triplicate column in the future Dfs
trip_df_marker=pd.DataFrame()
trip_df_marker["Well"]=well_list
trip_df_marker["Triplicate_#"]=trip_marker_list

In [52]:
#8
#This creates a dataframe for the negatives per plate
#empty dataframe
neg_finder_df=pd.DataFrame()

#adding two columns created earlier, donor list and well list
neg_finder_df["Donor_number"]=donor_name_list
neg_finder_df["Well"]=Master_neg_list

#creating a triplicate number for these donors, starting with an empty list
triplicate_fordonor_list=[]

#for loop based on specific well locations
for donorwell in neg_finder_df["Well"]:
    
    #only showing the unique well locations so no repeats
    special_list=trip_df_marker["Well"].unique()
    
    #if the specific well location is in the unique list then...
    if donorwell in  special_list:
        
        # then create a small DF on that donorwell
        special_df=trip_df_marker.loc[(trip_df_marker["Well"]==donorwell)]
        triplicate_fordonor_list.append(special_df)
        
#attach all the individual DF together into one and on each well        
hopefully_neg_df=pd.concat(triplicate_fordonor_list)
specialized_neg_df=pd.merge(neg_finder_df, hopefully_neg_df, on="Well")

In [53]:
#9
#applies the negative DF to the Triplicate DF
#these will be the names for the columns of the new DF
New_headers_for_trip=["Donor_number", "Triplicate_#"]
neg_df_trip_list=list(specialized_neg_df["Triplicate_#"])
neg_df_donornum_list=list(specialized_neg_df["Donor_number"])

Triplicate_forDonor=trip_df_marker["Triplicate_#"].unique()

Donor_number_by_triplicate=[]

#for loop based on the triplicate DF 
for Tnumber in Triplicate_forDonor:
    
    #if that number is greater than the biggest triplicate number of the last donor in the negative DF, then...
    if Tnumber>neg_df_trip_list[-1]:
        
        #...no donor is associated with it
        Donor_number_by_triplicate.append(None)
    
    #the else statement will now be dealing with the triplicates that are associated with donors
    else:
        
        #another counting while loop
        i = 0 
        while i < (len(specialized_neg_df["Triplicate_#"])) :
            
            #This is why the well order is important for input.
            #This will start with the first negative triplicate put in, which needs to be the first one encountered on the plate
            if Tnumber <= neg_df_trip_list[i]:
                    Donor_number_by_triplicate.append(neg_df_donornum_list[i])
                    
                    #this break prevents an endless loop from happening
                    break
            
            #The count will only occur once every triplicate that is less than the first donor triplicate is reached
            else:
                i=i+ 1
                
#Finishing up incorporating donor number into the triplicate DF
Donor_number_and_triplicate_df = pd.DataFrame(list(zip(Donor_number_by_triplicate, Triplicate_forDonor)), columns =New_headers_for_trip)
Trip_donornum_df=pd.merge(Donor_number_and_triplicate_df,trip_df_marker,how="inner",on=("Triplicate_#"))

In [54]:
#10
#This box changes the triplicate column from int to str so I can call on negative wells by name
Neg_trip_list=[]

#isolating only the unique well numbers, so no repeats
Trip_forneg_list=Trip_donornum_df["Well"].unique()

#for loop using the unique wells
for neg_well in Trip_forneg_list:
    i=0
    
    #this while loop is going to use all the negatives regardless of donor number to make the triplicates into "neg"
    while i < (len(flat_neg_list)):
        if neg_well ==flat_neg_list[i]:
            each_well_df=Trip_donornum_df.loc[(Trip_donornum_df["Well"]==neg_well)]
            Neg_trip_list.append(int(each_well_df["Triplicate_#"].unique()))
            break
        else:
            i=i+1

Trip_donornum_completed_df=Trip_donornum_df.replace(Neg_trip_list,"neg")

In [55]:
#11
#This list is what each column header should be when iris creates the file.
Expected_Cols=["Plate", "Well","Read Date", "Saved Date", "Machine ID", "Analyte Secreting Population", "LED Filter", "Spot Forming Units (SFU)"]

#This object is the datafile list after the function check_headers has been run.
Col_headers=check_headers(df_list)

#This for loop checks if each header within the excel files are what should be expected from the iris output.
for col_head in Col_headers:
    
    #If the headers are other than what is expected, there is an excel file that is not an iris output or is a broken iris excel file.
    assert col_head == Expected_Cols, "Column headers within one or more excel files are not as expected: "+col_head

In [56]:
#12
#Combines all dataframes made above into one large dataframe and removes the three columns below.
complete_df=(pd.concat(df_list, axis=0)).drop(columns=["Machine ID","Saved Date","Analyte Secreting Population"])
  
LEDcombinations=[]

#This isolates the different LED filters that the Mabtech puts out.
#This will be used to ensure that no matter how many Fluorospot colors you used (double, triple, etc.), the following script will gather all the data for them.
LED_list=complete_df["LED Filter"].unique()
for LED in LED_list:
    LEDcombinations.append(LED)

#This prints out the list of unique found combinations found within the LED Filter column of the Mabtech excel output.
#If it is not what you expected, please contact me.
print(LEDcombinations)

#This prints out the number of times a specific plate name shows up.
#This is helpful in determining where something may have went wrong
collections.Counter(complete_df["Plate"])

['LED490 Total', 'LED490 Single', 'LED550 Total', 'LED550 Single', 'LED640 Total', 'LED640 Single', 'LED490+LED640', 'LED550+LED640', 'LED490+LED550', 'LED490+LED550+LED640']


Counter({'DonorTP0030_Pool0238_Run01': 960,
         'DonorTP0031_Pool0054_Run01': 960,
         'DonorTP0012_Pool0570_Run01': 960,
         'DonorTP0031_Pool0780_Run01': 960,
         'DonorTP0012_Pool1047_Run01': 960,
         'DonorTP0031_Pool0243_Run01': 960,
         'DonorTP0012_Pool0190_Run01': 960,
         'DonorTP0031_Pool0154_Run01': 960,
         'DonorTP0012_Pool0554_Run01': 960,
         'DonorTP0012_Pool1045_Run01': 960,
         'DonorTP0012_Pool0961_Run01': 960,
         'DonorTP0012_Pool0218_Run01': 960,
         'DonorTP0031_Pool0723_Run01': 960,
         'DonorTP0012_Pool0805_Run01': 960,
         'DonorTP0012_Pool1046_Run01': 960,
         'DonorTP0030_Pool1066_Run01': 960,
         'DonorTP0031_Pool0215_Run01': 960,
         'DonorTP0031_Pool0029_Run01': 960,
         'DonorTP0031_Pool0168_Run01': 960,
         'DonorTP0031_Pool0864_Run01': 960,
         'DonorTP0012_Pool0039_Run01': 960})

In [57]:
#13
#Each of these empty lists will be filled based on the specific LED Filter only.
#Though only the Total df list and Single combo df list will be used in the rest of the script, I parcel out the singles and combo into their own list in case theyre needed.
Total_dfs_list=[]
Singles_list=[]
Combo_dfs_list=[]
Single_combo_dfs_list=[]
LEDCOMBO_headers=["Plate","Read Date","Donor_number","Well","Triplicate_#"]
#This defines an object headers, which will be used later.
headers=["Plate", "Well","Read Date","LED Filter"]

#This for loop is using the unique list defined in the previous box to ensure all the different combinations are incorporated.
for led in LEDcombinations:
    
    #The totals will be caught in this loop then added to their appropriate list.
    if led.endswith("Total"):
        total_df = (complete_df.loc[complete_df["LED Filter"]==led])
        Total_dfs_list.append(total_df)
    
    #The combinations will be caught in this loop then added to their appropriate lists.
    if "+" in led:
        
        #This both defines the df by its LED filter and renames the SFU column to its LED filter (useful later).
        combo_df=(complete_df.loc[complete_df["LED Filter"]==led])
        combo_df_for_combined_df = ((complete_df.loc[complete_df["LED Filter"]==led]).rename(columns={"Spot Forming Units (SFU)": led}))
        Combo_dfs_list.append(combo_df)
        Single_combo_dfs_list.append(combo_df_for_combined_df)
        LEDCOMBO_headers.append(led)
    
    #The singles will be caught in this loop then added to their appropriate lists.
    if led.endswith("Single"):
        
        #This both defines the df by its LED filter and renames the SFU column to its LED filter (useful later).
        single_df=(complete_df.loc[complete_df["LED Filter"]==led])
        singles_for_combined_df=((complete_df.loc[complete_df["LED Filter"]==led]).rename(columns={"Spot Forming Units (SFU)": led}))
        Singles_list.append(single_df)
        Single_combo_dfs_list.append(singles_for_combined_df)
        LEDCOMBO_headers.append(led)
        
LEDCOMBO_headers.append("Sum_of_all_activation")

In [58]:
#14
#This box containes a bit of magic I dont fully comprehend, but it combines dataframes based on their headers and LED filters.
from functools import reduce
single_combo_header=["Plate","Well","Read Date"]
New_single_combo_dfs_list=[]

#You will see this If loop a few times.
#If you are running a single-color fluorospot, this loop will skip certain analysis which would break the script.
if len(LEDcombinations) <2:
    pass
else:
    for df in Single_combo_dfs_list:
        new_df=df.drop(columns="LED Filter")
        New_single_combo_dfs_list.append(new_df)
    
    #The magic line is below. The merge will combine the DFs, but lambda and reduce remove repetitive columns to make the DF cleaner
    singles_and_combined_df=reduce(lambda x,y : pd.merge(x, y, on = single_combo_header), New_single_combo_dfs_list)

    #This sums up the singles and columns into a new sum row.
    sum_column=singles_and_combined_df.sum(axis=1)

    singles_and_combined_df["Sum_of_all_activation"]=sum_column

    #This runs the triplicate function on the newly created DF.
    triplicate_singles_and_combined_df=pd.merge(singles_and_combined_df,Trip_donornum_completed_df,how="outer",on=("Well"))

    All_Activation_Summary=triplicate_singles_and_combined_df[LEDCOMBO_headers]

    #Now that the Df is made and the new columns are added, this saves it to the master excel sheet.
    All_Activation_Summary.to_excel(writer,sheet_name="All_Activation_Summary")

In [59]:
#15
#This step is repeated three times for three different lists of DFs.
#A new DF is created based on the triplicates that get assigned by the function created in box #6.
trip_Total_dfs_list=[]
for df in Total_dfs_list:
    new_df=pd.merge(df,Trip_donornum_completed_df,how="outer",on=("Well"))
    trip_Total_dfs_list.append(new_df)

In [60]:
#16
trip_single_dfs_list=[]
if len(LEDcombinations) <2:
    pass
else:
    for df in Singles_list:
        new_df=pd.merge(df,Trip_donornum_completed_df,how="outer",on=("Well"))
        trip_single_dfs_list.append(new_df)

In [61]:
#17
trip_combo_dfs_list=[]
if len(LEDcombinations) <2:
    pass
else:
    for df in Combo_dfs_list:
        new_df=pd.merge(df,Trip_donornum_completed_df,how="outer",on=("Well"))
        trip_combo_dfs_list.append(new_df)

In [62]:
#18
#This is creating headers based on the number of negative wells you defined in box 2.
rep_str="rep"
Number_of_negative_wells = len(list_of_negs[0])
#This makes a list of the word "rep" for every negative well you defined.
rep_list=[rep_str]*Number_of_negative_wells
num_rep_list=[]

#This for loop will add numbers next to each rep in the list above.
for i in range(0, (len(rep_list))):
    num_rep_list.append("rep"+str(i+1))
print(num_rep_list)

['rep1', 'rep2', 'rep3']


In [63]:
#19
#This function will be used to isolate specific datum in future triplicates.
def get_replicates_from_row(df):
    replicates=[]
    for replicate_header in num_rep_list:
        for sfu in df[replicate_header]:
            replicates.append(sfu)
        
    return(replicates)

In [64]:
#20 
#This step gets repeated three times to three different lists of DFs

#This is a new a list of new col headers which will be used for a new df.
New_col=["Plate_ID", "Read_Date","Donor_number", "Triplicate_#","LED_Filter"]

#This is a repeat of the rep headers, but it adds it directly to the new headers above
rep_str="rep"
rep_list=[rep_str]*Number_of_negative_wells
for i in range(0, (len(rep_list))):
    New_col.append("rep"+str(i+1))
print(New_col)
#This empty list will be filled with the data beneath the new headers.
new_Total_dfs_list=[]

#For loop within the list of total dataframes.
for df in trip_Total_dfs_list:
    new_total_df_list=[]
    
    #This isolates data based on LED Filter then creates a smaller DF with that data.
    LED_list=df["LED Filter"].unique()
    for ledfilter in LED_list:
        ledfilter_df=df.loc[(df["LED Filter"]==ledfilter)]
        
        #This isolates data within the smaller DF based on Plate name then creates a smaller DF with that data.
        plate_list=ledfilter_df["Plate"].unique()
        for plate in plate_list:
            plate_df = df.loc[(df['Plate']==plate)]

            #This isolates data within the even smaller DF based on Read date then creates a smaller DF with that data.
            Read_date_list=plate_df["Read Date"].unique()
            for read_date in Read_date_list:
                read_date_df= plate_df.loc[(plate_df["Read Date"]==read_date)]
                
                donor_num_list=read_date_df["Donor_number"].unique()
                for donor in donor_num_list:
                    donornum_df=read_date_df.loc[(read_date_df["Donor_number"]==donor)]
                    
                    
                    #This isolates data within the read date DF based on Triplicate # then creates a smaller DF with that data.
                    trip_list=donornum_df["Triplicate_#"].unique()
                    for trip in trip_list:
                        trips_df=donornum_df.loc[(donornum_df["Triplicate_#"]==trip)]

                        #Now the tiny DF will be segmented based on the pieces used to isolate the DF.
                        new_rows = [ plate, read_date, donor, trip, ledfilter ]

                        #What is within the tiny DF is a unique set of 3 or 6 SFUs per LED, Plate, Read Date, Triplicate #.
                        #This for loop will now run through and isolate each SFU by row and name.
                        for index, row in trips_df.iterrows():
                            
                            #With these isolated SFUs, a new object is formed.
                            SFU=row["Spot Forming Units (SFU)"]
    
                            #This object is now added to the new row set made above.
                            new_rows.append( SFU )

                        #The new row is then added into a list to become a new df.
                        new_total_df_list.append(new_rows)

        #This is taking the list of new rows and making them into a new df and giving them the new headers defined int he beginning.
        new_total_df=pd.DataFrame( new_total_df_list , columns = New_col )
        
        #This adds the new DF to a list of all the new Total Dfs.
        new_Total_dfs_list.append(new_total_df)

['Plate_ID', 'Read_Date', 'Donor_number', 'Triplicate_#', 'LED_Filter', 'rep1', 'rep2', 'rep3']


In [65]:
#21
#This empty list will be filled with the data beneath the new headers.
new_single_dfs_list=[]
if len(LEDcombinations) <2:
    pass
else:
    #For loop within the list of total dataframes.
    for df in trip_single_dfs_list:
        new_single_df_list=[]

        #This isolates data based on LED Filter then creates a smaller DF with that data.
        LED_list=df["LED Filter"].unique()
        for ledfilter in LED_list:
            ledfilter_df=df.loc[(df["LED Filter"]==ledfilter)]

            #This isolates data within the smaller DF based on Plate name then creates a smaller DF with that data.
            plate_list=ledfilter_df["Plate"].unique()
            for plate in plate_list:
                plate_df = df.loc[(df['Plate']==plate)]

                #This isolates data within the even smaller DF based on Read date then creates a smaller DF with that data.
                Read_date_list=plate_df["Read Date"].unique()
                for read_date in Read_date_list:
                    read_date_df= plate_df.loc[(plate_df["Read Date"]==read_date)]
                    
                    donor_num_list=read_date_df["Donor_number"].unique()
                    for donor in donor_num_list:
                        donornum_df=read_date_df.loc[(read_date_df["Donor_number"]==donor)]
                    
                        #This isolates data within the read date DF based on Triplicate # then creates a smaller DF with that data.
                        trip_list=donornum_df["Triplicate_#"].unique()
                        for trip in trip_list:
                            trips_df=donornum_df.loc[(donornum_df["Triplicate_#"]==trip)]

                            #Now the tiny DF will be segmented based on the pieces used to isolate the DF.
                            new_rows = [ plate, read_date, donor, trip, ledfilter ]

                            #What is within the tiny DF is a unique set of 3 or 6 SFUs per LED, Plate, Read Date, Triplicate #.
                            #This for loop will now run through and isolate each SFU by row and name.
                            for index, row in trips_df.iterrows():

                                #With these isolated SFUs, a new object is formed.
                                SFU=row["Spot Forming Units (SFU)"]

                                #This object is now added to the new row set made above.
                                new_rows.append( SFU )

                            #The new row is then added into a list to become a new df.
                            new_single_df_list.append(new_rows)

            #This is taking the list of new rows and making them into a new df and giving them the new headers defined int he beginning.
            new_single_df=pd.DataFrame( new_single_df_list , columns = New_col )

            #This adds the new DF to a list of all the new Total Dfs.
            new_single_dfs_list.append(new_single_df)

In [66]:
#22
#This empty list will be filled with the data beneath the new headers.
new_combo_dfs_list=[]
if len(LEDcombinations) <2:
    pass
else:
    #For loop within the list of total dataframes.
    for df in trip_combo_dfs_list:
        new_combo_df_list=[]

        #This isolates data based on LED Filter then creates a smaller DF with that data.
        LED_list=df["LED Filter"].unique()
        for ledfilter in LED_list:
            ledfilter_df=df.loc[(df["LED Filter"]==ledfilter)]

            #This isolates data within the smaller DF based on Plate name then creates a smaller DF with that data.
            plate_list=ledfilter_df["Plate"].unique()
            for plate in plate_list:
                plate_df = df.loc[(df['Plate']==plate)]

                #This isolates data within the even smaller DF based on Read date then creates a smaller DF with that data.
                Read_date_list=plate_df["Read Date"].unique()
                for read_date in Read_date_list:
                    read_date_df= plate_df.loc[(plate_df["Read Date"]==read_date)]

                    donor_num_list=read_date_df["Donor_number"].unique()
                    for donor in donor_num_list:
                        donornum_df=read_date_df.loc[(read_date_df["Donor_number"]==donor)]
                    
                        #This isolates data within the read date DF based on Triplicate # then creates a smaller DF with that data.
                        trip_list=donornum_df["Triplicate_#"].unique()
                        for trip in trip_list:
                            trips_df=donornum_df.loc[(donornum_df["Triplicate_#"]==trip)]

                            #Now the tiny DF will be segmented based on the pieces used to isolate the DF.
                            new_rows = [ plate, read_date, donor, trip, ledfilter ]

                            #What is within the tiny DF is a unique set of 3 or 6 SFUs per LED, Plate, Read Date, Triplicate #.
                            #This for loop will now run through and isolate each SFU by row and name.
                            for index, row in trips_df.iterrows():

                                #With these isolated SFUs, a new object is formed.
                                SFU=row["Spot Forming Units (SFU)"]

                                #This object is now added to the new row set made above.
                                new_rows.append( SFU )

                            #The new row is then added into a list to become a new df.
                            new_combo_df_list.append(new_rows)

            #This is taking the list of new rows and making them into a new df and giving them the new headers defined int he beginning.
            new_combo_df=pd.DataFrame( new_combo_df_list , columns = New_col )

            #This adds the new DF to a list of all the new Total Dfs.
            new_combo_dfs_list.append(new_combo_df)

In [67]:
#23
#This box is also repeated three times for three different lists of DFs.
#This box takes the average and standard deviation of each triplicate then adds it to a new column in the DF.
for df in new_Total_dfs_list:
    new_total_df_Average=df.mean(axis=1, skipna=True)
    new_total_df_StDev=df.std(axis=1, skipna=True)
    
    df["Average"]=new_total_df_Average
    df["StDev"]=new_total_df_StDev

In [68]:
#24
#This box takes the average and standard deviation of each triplicate then adds it to a new column in the DF.
if len(LEDcombinations) <2:
    pass
else:
    for df in new_single_dfs_list:
        new_single_df_Average=df.mean(axis=1, skipna=True)
        new_single_df_StDev=df.std(axis=1, skipna=True)

        df["Average"]=new_single_df_Average
        df["StDev"]=new_single_df_StDev

In [69]:
#25
#This box takes the average and standard deviation of each triplicate then adds it to a new column in the DF.
if len(LEDcombinations) <2:
    pass
else: 
    combo_header=["Plate_ID","Read_Date","Donor_number","Triplicate_#"]
    for df in new_combo_dfs_list:
        new_combo_df_Average=df.mean(axis=1, skipna=True)
        new_combo_df_StDev=df.std(axis=1, skipna=True)

        df["Average"]=new_combo_df_Average
        df["StDev"]=new_combo_df_StDev

    new_combo_df=reduce(lambda x,y : pd.merge(x, y, on = combo_header), new_combo_dfs_list)
    new_combo_df.to_excel(writer,sheet_name="Just_double_triple_producers")

In [70]:
#26
#This essentially defines the poisson function to be called later.
from scipy.stats import poisson

In [71]:
#27
#This essentially defines a function as stats to be called later.
import math
from scipy import stats

In [72]:
#28
#Using the number of cells you defined per plate earlier, this function creates the net sfu number from an average.
def get_netsfu(average):
    num_to_get_to_mil=1000000/Number_of_cells_per_well
    net_sfu=((average-Neg_Avg)*num_to_get_to_mil)
    if net_sfu > 0:
        return(net_sfu)
    if net_sfu<0:
        return(0)

In [73]:
#29
#This box runs several equations and creates multiple columns.

#A lot of what is happening is similar to box 13, so I only explain the new parts.
statistics_Total_dfs_list=[]
for df in new_Total_dfs_list:
    total_NetSFC=[]
    total_SI_list=[]
    total_poisson_Rep1_list=[]
    total_poisson_Rep2_list=[]
    total_poisson_Rep3_list=[]

    plate_list=df["Plate_ID"].unique()
    for plate in plate_list:
        plate_df = df.loc[(df['Plate_ID']==plate)]
    
        read_date_list=plate_df["Read_Date"].unique()
        for read_date in read_date_list:
            read_date_df=plate_df.loc[(plate_df["Read_Date"]==read_date)]
            
            donor_num_list=read_date_df["Donor_number"].unique()
            for donor in donor_num_list:
                donornum_df=read_date_df.loc[(read_date_df["Donor_number"]==donor)]
                    
                #Using the read date DF, the negative averages will be made.
                Neg_df=donornum_df.loc[(read_date_df["Triplicate_#"]=="neg")]
                
                Neg_Avg=float(Neg_df["Average"])

                #If the neg average is above or equal to 2, mu for poisson will use it.
                if Neg_Avg>=2:
                    mu=Neg_Avg

                #Otherwise mu will be defined as 2.
                else:
                    mu=2

                trip_list=donornum_df["Triplicate_#"].unique()
                for trip in trip_list:
                    trips_df=donornum_df.loc[(donornum_df["Triplicate_#"]==trip)]

                    Average_list=trips_df["Average"]
                    for average in Average_list:

                        #This runs the netsfu function then uses it to make the SI
                        netsfc=get_netsfu(average)
                        total_NetSFC.append(netsfc)

                        #As long as the Neg average is above 0, the average of each triplicate will be divided by it.
                        if Neg_Avg > 0:
                            SI=(average/Neg_Avg)
                            total_SI_list.append(SI)

                        #Otherwise SI is just the average of the triplicate.
                        else:
                            zero_neg_SI=average
                            total_SI_list.append(zero_neg_SI)

                    Rep1=trips_df["rep1"]
                    #The next for loops are running poisson.
                    for rep1 in Rep1:
                        if rep1>=1:
                            x1=rep1-1
                        else:
                            x1=0

                        poisson1="{:.1%}".format(1-stats.poisson.cdf(x1,mu,loc=0))
                        total_poisson_Rep1_list.append(poisson1)

                    Rep2=trips_df["rep2"]
                    for rep2 in Rep2:
                        if rep2>=1:
                            x2=rep2-1
                        else:
                            x2=0

                        poisson2="{:.1%}".format(1-stats.poisson.cdf(x2,mu,loc=0))
                        total_poisson_Rep2_list.append(poisson2)

                    Rep3=trips_df["rep3"]
                    for rep3 in Rep3:
                        if rep3>=1:
                            x3=rep3-1
                        else:
                            x3=0

                        poisson3="{:.1%}".format(1-stats.poisson.cdf(x3,mu,loc=0))
                        total_poisson_Rep3_list.append(poisson3)

    #With the data gathered above, new columns are created with the information.
    df["NetSFC_per_million"]=total_NetSFC
    df["SI"]=total_SI_list
    df["Poisson_Rep1"]=total_poisson_Rep1_list
    df["Poisson_Rep2"]=total_poisson_Rep2_list
    df["Poisson_Rep3"]=total_poisson_Rep3_list
    statistics_Total_dfs_list.append(df)


In [74]:
#30
#This box runs several equations and creates multiple columns.

#A lot of what is happening is similar to box 13, so I only explain the new parts.
if len(LEDcombinations) <2:
    pass
else:  
    statistics_single_dfs_list=[]
    for df in new_single_dfs_list:
        single_NetSFC=[]
        single_SI_list=[]
        single_poisson_Rep1_list=[]
        single_poisson_Rep2_list=[]
        single_poisson_Rep3_list=[]

        plate_list=df["Plate_ID"].unique()
        for plate in plate_list:
            plate_df = df.loc[(df['Plate_ID']==plate)]

            read_date_list=plate_df["Read_Date"].unique()
            for read_date in read_date_list:
                read_date_df=plate_df.loc[(plate_df["Read_Date"]==read_date)]

                donor_num_list=read_date_df["Donor_number"].unique()
                for donor in donor_num_list:
                    donornum_df=read_date_df.loc[(read_date_df["Donor_number"]==donor)]

                    #Using the read date DF, the negative averages will be made.
                    Neg_df=donornum_df.loc[(read_date_df["Triplicate_#"]=="neg")]

                    Neg_Avg=float(Neg_df["Average"])

                    #If the neg average is above or equal to 2, mu for poisson will use it.
                    if Neg_Avg>=2:
                        mu=Neg_Avg

                    #Otherwise mu will be defined as 2.
                    else:
                        mu=2

                    trip_list=donornum_df["Triplicate_#"].unique()
                    for trip in trip_list:
                        trips_df=donornum_df.loc[(donornum_df["Triplicate_#"]==trip)]

                        Average_list=trips_df["Average"]
                        for average in Average_list:

                            #This runs the netsfu function then uses it to make the SI
                            netsfc=get_netsfu(average)
                            single_NetSFC.append(netsfc)

                            #As long as the Neg average is above 0, the average of each triplicate will be divided by it.
                            if Neg_Avg > 0:
                                SI=(average/Neg_Avg)
                                single_SI_list.append(SI)

                            #Otherwise SI is just the average of the triplicate.
                            else:
                                zero_neg_SI=average
                                single_SI_list.append(zero_neg_SI)

                        Rep1=trips_df["rep1"]
                        #The next for loops are running poisson.
                        for rep1 in Rep1:
                            if rep1>=1:
                                x1=rep1-1
                            else:
                                x1=0

                            poisson1="{:.1%}".format(1-stats.poisson.cdf(x1,mu,loc=0))
                            single_poisson_Rep1_list.append(poisson1)

                        Rep2=trips_df["rep2"]
                        for rep2 in Rep2:
                            if rep2>=1:
                                x2=rep2-1
                            else:
                                x2=0

                            poisson2="{:.1%}".format(1-stats.poisson.cdf(x2,mu,loc=0))
                            single_poisson_Rep2_list.append(poisson2)

                        Rep3=trips_df["rep3"]
                        for rep3 in Rep3:
                            if rep3>=1:
                                x3=rep3-1
                            else:
                                x3=0

                            poisson3="{:.1%}".format(1-stats.poisson.cdf(x3,mu,loc=0))
                            single_poisson_Rep3_list.append(poisson3)

        #With the data gathered above, new columns are created with the information.
        df["NetSFC_per_million"]=single_NetSFC
        df["SI"]=single_SI_list
        df["Poisson_Rep1"]=single_poisson_Rep1_list
        df["Poisson_Rep2"]=single_poisson_Rep2_list
        df["Poisson_Rep3"]=single_poisson_Rep3_list
        statistics_single_dfs_list.append(df)

In [75]:
#31
#This little function is used to make the ttest result reflect a one tailed result instead of a two tailed one.
def get_pvalue(self):
        
        return (self[1]/2)

In [76]:
#32
#Most of this box you've seen in prior boxes.
tt_total_dfs_list=[]
for df in statistics_Total_dfs_list:   
    
    total_ttests=[]

    plate_list=df["Plate_ID"].unique()
    for plate in plate_list:
        plate_df = df.loc[(df['Plate_ID']==plate)]
    
        read_date_list=plate_df["Read_Date"].unique()
        for read_date in read_date_list:
            read_date_df=plate_df.loc[(plate_df["Read_Date"]==read_date)]
            
            donor_num_list=read_date_df["Donor_number"].unique()
            for donor in donor_num_list:
                donornum_df=read_date_df.loc[(read_date_df["Donor_number"]==donor)]


                Neg_triplicates=[]
                Neg_df=donornum_df.loc[(donornum_df["Triplicate_#"]=="neg")]
                Neg_triplicates=get_replicates_from_row(Neg_df)

                trip_list=donornum_df["Triplicate_#"].unique()
                for trip in trip_list:
                    trips_df=donornum_df.loc[(donornum_df["Triplicate_#"]==trip)]
                    triplicates=get_replicates_from_row(trips_df)

                    #This will get the Ttest p-value from each triplicate
                    ttest_result=get_pvalue(stats.ttest_ind(triplicates, Neg_triplicates, equal_var = False, nan_policy='omit'))
                    total_ttests.append(ttest_result)


    df["Ttest_pvalue"]=total_ttests
    tt_total_dfs_list.append(df)          

In [77]:
#33
#Most of this box you've seen in prior boxes.
if len(LEDcombinations) <2:
    pass
else: 
    tt_single_dfs_list=[]
    for df in statistics_single_dfs_list:   

        single_ttests=[]

        plate_list=df["Plate_ID"].unique()
        for plate in plate_list:
            plate_df = df.loc[(df['Plate_ID']==plate)]

            read_date_list=plate_df["Read_Date"].unique()
            for read_date in read_date_list:
                read_date_df=plate_df.loc[(plate_df["Read_Date"]==read_date)]
                
                donor_num_list=read_date_df["Donor_number"].unique()
                for donor in donor_num_list:
                    donornum_df=read_date_df.loc[(read_date_df["Donor_number"]==donor)]


                    Neg_triplicates=[]
                    Neg_df=donornum_df.loc[(donornum_df["Triplicate_#"]=="neg")]
                    Neg_triplicates=get_replicates_from_row(Neg_df)

                    trip_list=donornum_df["Triplicate_#"].unique()
                    for trip in trip_list:
                        trips_df=donornum_df.loc[(donornum_df["Triplicate_#"]==trip)]
                        triplicates=get_replicates_from_row(trips_df)

                        #This will get the Ttest p-value from each triplicate
                        ttest_result=get_pvalue(stats.ttest_ind(triplicates, Neg_triplicates, equal_var = False, nan_policy='omit'))
                        single_ttests.append(ttest_result)


        df["Ttest_pvalue"]=single_ttests
        tt_single_dfs_list.append(df)          

In [78]:
#34
#Another import of stats here, its odd but I found it to be necessary.
import statistics

In [79]:
#35
#This box creates a new column that automatically checks if the triplicate is a positive hit based on specifications defined within.

#These specifications can be tinkered with, but it is sort of finicky.
positive_total_dfs_list=[]
for df in statistics_Total_dfs_list:   

    total_positive=[]
    plate_list=df["Plate_ID"].unique()
    for plate in plate_list:
        plate_df = df.loc[(df['Plate_ID']==plate)]
    
        read_date_list=plate_df["Read_Date"].unique()
        for read_date in read_date_list:
            read_date_df=plate_df.loc[(plate_df["Read_Date"]==read_date)]
                
            donor_num_list=read_date_df["Donor_number"].unique()
            for donor in donor_num_list:
                donornum_df=read_date_df.loc[(read_date_df["Donor_number"]==donor)]


                trip_list=donornum_df["Triplicate_#"].unique()
                for trip in trip_list:
                    trips_df=donornum_df.loc[(donornum_df["Triplicate_#"]==trip)]

                    for sfc in trips_df["NetSFC_per_million"]:

                        #This will check the sfc and if its over the cutoff you defined, it will move to the next step. Otherwise it will be labelled negative.
                        if sfc >=Number_of_sfcs_necessary_to_be_considered:
                            for si in trips_df["SI"]:

                                #If the triplicate's si is greater than 2, it will move to the next step. Otherwise it will be labelled as negative.
                                if si>=2:

                                    #These next few steps will create an average for the poissons.
                                    rep_list=[]
                                    for REP1 in trips_df["Poisson_Rep1"]:
                                        r1=float(REP1.replace("%",""))
                                        rep_list.append(r1)
                                    for REP2 in trips_df["Poisson_Rep2"]:
                                        r2=float(REP2.replace("%",""))
                                        rep_list.append(r2)
                                    for REP3 in trips_df["Poisson_Rep3"]:
                                        r3=float(REP3.replace("%",""))
                                        rep_list.append(r3)
                                    Poisson_avg=statistics.mean(rep_list)

                                    #Now the pvalue cutoffs will be checked.
                                    for pvalue in trips_df["Ttest_pvalue"]:

                                        #If the pvalue of the ttest is less than .05 OR the poisson avg found above is less than 5, the sfc # will be put into the new positive column.
                                        if pvalue <=.05 or Poisson_avg <= 5:
                                            t=sfc
                                            total_positive.append(t)

                                        #Otherwise, it will be labelled negative in the positive column.
                                        else:
                                            f="Negative"
                                            total_positive.append(f)
                                else:
                                    f="Negative"
                                    total_positive.append(f)
                        else:
                            f="Negative"
                            total_positive.append(f)

    df["Positive"]=total_positive
    positive_total_dfs_list.append(df) 

In [80]:
#36
#This box creates a new column that automatically checks if the triplicate is a positive hit based on specifications defined within.

#These specifications can be tinkered with, but it is sort of finicky.
if len(LEDcombinations) <2:
    pass
else: 
    positive_single_dfs_list=[]
    for df in statistics_single_dfs_list:   

        single_positive=[]
        plate_list=df["Plate_ID"].unique()
        for plate in plate_list:
            plate_df = df.loc[(df['Plate_ID']==plate)]

            read_date_list=plate_df["Read_Date"].unique()
            for read_date in read_date_list:
                read_date_df=plate_df.loc[(plate_df["Read_Date"]==read_date)]

                donor_num_list=read_date_df["Donor_number"].unique()
                for donor in donor_num_list:
                    donornum_df=read_date_df.loc[(read_date_df["Donor_number"]==donor)]


                    trip_list=donornum_df["Triplicate_#"].unique()
                    for trip in trip_list:
                        trips_df=donornum_df.loc[(donornum_df["Triplicate_#"]==trip)]

                        for sfc in trips_df["NetSFC_per_million"]:

                            #This will check the sfc and if its over the cutoff you defined, it will move to the next step. Otherwise it will be labelled negative.
                            if sfc >=Number_of_sfcs_necessary_to_be_considered:
                                for si in trips_df["SI"]:

                                    #If the triplicate's si is greater than 2, it will move to the next step. Otherwise it will be labelled as negative.
                                    if si>=2:

                                        #These next few steps will create an average for the poissons.
                                        rep_list=[]
                                        for REP1 in trips_df["Poisson_Rep1"]:
                                            r1=float(REP1.replace("%",""))
                                            rep_list.append(r1)
                                        for REP2 in trips_df["Poisson_Rep2"]:
                                            r2=float(REP2.replace("%",""))
                                            rep_list.append(r2)
                                        for REP3 in trips_df["Poisson_Rep3"]:
                                            r3=float(REP3.replace("%",""))
                                            rep_list.append(r3)
                                        Poisson_avg=statistics.mean(rep_list)

                                        #Now the pvalue cutoffs will be checked.
                                        for pvalue in trips_df["Ttest_pvalue"]:

                                            #If the pvalue of the ttest is less than .05 OR the poisson avg found above is less than 5, the sfc # will be put into the new positive column.
                                            if pvalue <=.05 or Poisson_avg <= 5:
                                                t=sfc
                                                single_positive.append(t)

                                            #Otherwise, it will be labelled negative in the positive column.
                                            else:
                                                f="Negative"
                                                single_positive.append(f)
                                    else:
                                        f="Negative"
                                        single_positive.append(f)
                            else:
                                f="Negative"
                                single_positive.append(f)

        df["Positive"]=single_positive
        positive_single_dfs_list.append(df) 

In [81]:
#37

#The final master df is made so that each LED Total DF is put next to eachother.
totals_headers=["Plate_ID","Read_Date","Donor_number","Triplicate_#"]

#Magic Line!
master_totals_combined_df=reduce(lambda x,y : pd.merge(x, y, on = totals_headers), positive_total_dfs_list)

#This saves the master Df to the new master excel file that will appear later.
master_totals_combined_df.to_excel(writer, sheet_name="Master_Totals_only")

In [82]:
#38
#The final master df is made so that each LED Total DF is put next to eachother.
if len(LEDcombinations) <2:
    pass
else:
    single_headers=["Plate_ID","Read_Date","Donor_number","Triplicate_#"]

    #Magic Line!
    master_singles_combined_df=reduce(lambda x,y : pd.merge(x, y, on = single_headers), positive_single_dfs_list)

    #This saves the master Df to the new master excel file that will appear later.
    master_singles_combined_df.to_excel(writer, sheet_name="Master_Singles_only")

In [83]:
#39
New_combined_col=["Plate_ID", "Read_Date", "Donor_number", "Triplicate_#"]

rep_str="rep"
rep_list=[rep_str]*Number_of_negative_wells
for i in range(0, (len(rep_list))):
    New_combined_col.append("rep"+str(i+1))

if len(LEDcombinations) <2:
    pass
else:
    new_singles_and_combined_df_list=[]

    plate_list1=All_Activation_Summary["Plate"].unique()
    for plate in plate_list1:
        plate_df = All_Activation_Summary.loc[(All_Activation_Summary['Plate']==plate)]

        Read_date_list=plate_df["Read Date"].unique()
        for read_date in Read_date_list:
            read_date_df= plate_df.loc[(plate_df["Read Date"]==read_date)]

            donor_num_list=read_date_df["Donor_number"].unique()
            for donor in donor_num_list:
                donornum_df=read_date_df.loc[(read_date_df["Donor_number"]==donor)]

                trip_list=donornum_df["Triplicate_#"].unique()
                for trip in trip_list:
                    trips_df=donornum_df.loc[(donornum_df["Triplicate_#"]==trip)]

                    #taking the unique triplet and joining them together into a list of unique rows
                    new_rows = [ plate, read_date, donor, trip ]

                    #what is left after the loops above is a unique set of 3 or 6 SFUs per donor, plate, and pool
                    #this will now be ran through iterrows, which will isolate each SFU by row and index
                    for index, row in trips_df.iterrows():

                        #with these isolated SFUs, a new object is formed
                        SFU=row["Sum_of_all_activation"]

                        #this object is now added to the new row set made above
                        new_rows.append( SFU )

                        #the new row can now be added into a list to become a new df
                
                    new_singles_and_combined_df_list.append(new_rows)

    #this is taking the new rows and making them into a new df
    new_singles_and_combined_df=pd.DataFrame( new_singles_and_combined_df_list , columns = New_combined_col )

In [84]:
#40
if len(LEDcombinations) <2:
    pass
else:
    new_singles_and_combined_df_Average=new_singles_and_combined_df.mean(axis=1, skipna=True)
    new_singles_and_combined_df_StDev=new_singles_and_combined_df.std(axis=1, skipna=True)

    new_singles_and_combined_df["Average"]=new_singles_and_combined_df_Average
    new_singles_and_combined_df["StDev"]=new_singles_and_combined_df_StDev

In [85]:
#41
NetSFC=[]
SI_list=[]
poisson_Rep1_list=[]
poisson_Rep2_list=[]
poisson_Rep3_list=[]
if len(LEDcombinations) <2:
    pass
else:
    plate_list2=new_singles_and_combined_df["Plate_ID"].unique()
    for plate in plate_list2:
        plate_df = new_singles_and_combined_df.loc[(new_singles_and_combined_df['Plate_ID']==plate)]


        read_date_list=plate_df["Read_Date"].unique()
        for read_date in read_date_list:
            read_date_df=plate_df.loc[(plate_df["Read_Date"]==read_date)]
           
            donor_num_list=read_date_df["Donor_number"].unique()
            for donor in donor_num_list:
                donornum_df=read_date_df.loc[(read_date_df["Donor_number"]==donor)]
                
                Neg_df=donornum_df.loc[(donornum_df["Triplicate_#"]=="neg")]
                Neg_Avg=float(Neg_df["Average"])

                if Neg_Avg>=2:
                    mu=Neg_Avg
                else:
                    mu=2

                trip_list=donornum_df["Triplicate_#"].unique()
                for trip in trip_list:
                    trips_df=donornum_df.loc[(donornum_df["Triplicate_#"]==trip)]


                    Average_list=trips_df["Average"]
                    for average in Average_list:

                        #taking each average minus the negative average of the pool then multiplying it by 5
                        netsfc=get_netsfu(average)
                        NetSFC.append(netsfc)

                        if Neg_Avg > 0:
                            SI=(average/Neg_Avg)
                            SI_list.append(SI)
                        else:
                            zero_neg_SI=average
                            SI_list.append(zero_neg_SI)

                    Rep1=trips_df["rep1"]
                    for rep1 in Rep1:
                        if rep1>=1:
                            x1=rep1-1
                        else:
                            x1=0

                        poisson1="{:.1%}".format(1-stats.poisson.cdf(x1,mu,loc=0))
                        poisson_Rep1_list.append(poisson1)

                    Rep2=trips_df["rep2"]
                    for rep2 in Rep2:
                        if rep2>=1:
                            x2=rep2-1
                        else:
                            x2=0

                        poisson2="{:.1%}".format(1-stats.poisson.cdf(x2,mu,loc=0))
                        poisson_Rep2_list.append(poisson2)

                    Rep3=trips_df["rep3"]
                    for rep3 in Rep3:
                        if rep3>=1:
                            x3=rep3-1
                        else:
                            x3=0

                        poisson3="{:.1%}".format(1-stats.poisson.cdf(x3,mu,loc=0))
                        poisson_Rep3_list.append(poisson3)

    new_singles_and_combined_df["NetSFC_per_million"]=NetSFC
    new_singles_and_combined_df["SI"]=SI_list
    new_singles_and_combined_df["Poisson_Rep1"]=poisson_Rep1_list
    new_singles_and_combined_df["Poisson_Rep2"]=poisson_Rep2_list
    new_singles_and_combined_df["Poisson_Rep3"]=poisson_Rep3_list

In [86]:
#42
ttests=[]
if len(LEDcombinations) <2:
    pass
else:
    plate_list3=new_singles_and_combined_df["Plate_ID"].unique()
    for plate in plate_list3:
        plate_df = new_singles_and_combined_df.loc[(new_singles_and_combined_df['Plate_ID']==plate)]

        read_date_list=plate_df["Read_Date"].unique()
        for read_date in read_date_list:
            read_date_df=plate_df.loc[(plate_df["Read_Date"]==read_date)]

            donor_num_list=read_date_df["Donor_number"].unique()
            for donor in donor_num_list:
                donornum_df=read_date_df.loc[(read_date_df["Donor_number"]==donor)]
            
                Neg_triplicates=[]
                Neg_df=donornum_df.loc[(donornum_df["Triplicate_#"]=="neg")]
                Neg_triplicates=get_replicates_from_row(Neg_df)

                trip_list=donornum_df["Triplicate_#"].unique()
                for trip in trip_list:
                    trips_df=donornum_df.loc[(donornum_df["Triplicate_#"]==trip)]
                    triplicates=get_replicates_from_row(trips_df)


                    ttest_result=get_pvalue(stats.ttest_ind(triplicates, Neg_triplicates, equal_var = False, nan_policy='omit'))
                    ttests.append(ttest_result)


    new_singles_and_combined_df["Ttest_pvalue"]=ttests        

In [87]:
#43
new_singles_and_combined_positive=[]
if len(LEDcombinations) <2:
    pass
else:
    plate_list4=new_singles_and_combined_df["Plate_ID"].unique()
    for plate in plate_list4:
        plate_df = new_singles_and_combined_df.loc[(new_singles_and_combined_df['Plate_ID']==plate)]


        read_date_list=plate_df["Read_Date"].unique()
        for read_date in read_date_list:
            read_date_df=plate_df.loc[(plate_df["Read_Date"]==read_date)]

            donor_num_list=read_date_df["Donor_number"].unique()
            for donor in donor_num_list:
                donornum_df=read_date_df.loc[(read_date_df["Donor_number"]==donor)]
            
                #repeating now for the unique pool responses in the plate df
                trip_list=donornum_df["Triplicate_#"].unique()
                for trip in trip_list:
                    trips_df=donornum_df.loc[(donornum_df["Triplicate_#"]==trip)]

                    for sfc in trips_df["NetSFC_per_million"]:
                        if sfc >=Number_of_sfcs_necessary_to_be_considered:
                            for si in trips_df["SI"]:
                                if si>=2:
                                    rep_list=[]
                                    for REP1 in trips_df["Poisson_Rep1"]:
                                        r1=float(REP1.replace("%",""))
                                        rep_list.append(r1)
                                    for REP2 in trips_df["Poisson_Rep2"]:
                                        r2=float(REP2.replace("%",""))
                                        rep_list.append(r2)
                                    for REP3 in trips_df["Poisson_Rep3"]:
                                        r3=float(REP3.replace("%",""))
                                        rep_list.append(r3)
                                    Poisson_avg=statistics.mean(rep_list)
                                    for pvalue in trips_df["Ttest_pvalue"]:
                                        if pvalue <=.05 or Poisson_avg <= 5:
                                            t=sfc
                                            new_singles_and_combined_positive.append(t)
                                        else:
                                            f="Negative"
                                            new_singles_and_combined_positive.append(f)
                                else:
                                    f="Negative"
                                    new_singles_and_combined_positive.append(f)
                        else:
                            f="Negative"
                            new_singles_and_combined_positive.append(f)

    new_singles_and_combined_df["Positive"]=new_singles_and_combined_positive
    new_singles_and_combined_df.to_excel(writer, sheet_name="Master_Singles+Combined")

In [88]:
#44
#This saves the Master Excel file to path defined in the writer function.
#If unchanged, it will end up on your desktop.
writer.save()